In [85]:
import os
import os.path
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
t = 450
thres = 4.5


In [86]:
x = []
y = []
file_cnt = 0
for dirpath, dirnames, filenames in os.walk("../Fall Data (New)/"):
    for filename in [f for f in filenames if f.endswith(".txt")]:
        df = pd.read_csv(os.path.join(dirpath, filename),header=None)
        fall_time = df[df[1].isna()]
        df = df[df[1].notna()]
        ax = np.array(df[1])
        ay = np.array(df[2])
        az = np.array(df[3])
        gx = np.array(df[4])
        gy = np.array(df[5])
        gz = np.array(df[6])
        time = np.array(df[0])
        a = np.sqrt(ax**2 + ay**2 + az**2)
        if filename[0] == "A":
            # ADL file
            file_cnt += 1
            for i in range(a.shape[0]):
                if a[i] >= thres:
                    st = i - t
                    ed =  i + t
                    if st < 0 or ed >= a.shape[0]:
                        continue

                    tmp = np.array(list(zip(ax[st:ed],ay[st:ed],az[st:ed],gx[st:ed],gy[st:ed],gz[st:ed])))


                    x.append(tmp)
                    y.append(1)
                    
        elif filename[11] == "B":
            i = np.argmax(a)
            st = i - t
            ed =  i + t
            if st < 0 or ed >= a.shape[0] or a[i] < thres:
                print(filename,st,ed,a[i])
                continue
            file_cnt += 1
            tmp = np.array(list(zip(ax[st:ed],ay[st:ed],az[st:ed],gx[st:ed],gy[st:ed],gz[st:ed])))
            x.append(tmp)
            y.append(2)
            print("Same floor Fall:",filename)
            continue

        else:
            # Fall Data
            i = np.argmax(a)
            st = i - t
            ed =  i + t
            if st < 0 or ed >= a.shape[0] or a[i] < thres:
                print(filename,st,ed,a[i])
                continue
            file_cnt += 1
            print("Fall:",filename)
            tmp = np.array(list(zip(ax[st:ed],ay[st:ed],az[st:ed],gx[st:ed],gy[st:ed],gz[st:ed])))
            x.append(tmp)
            if len(y) == 1157:
                print(i,filename)
            y.append(0)
            
print(file_cnt)

TESTING_ML_v10.txt -102 798 12.53990430585497
Fall: TESTING_ML_v15.txt
Fall: TESTING_ML_v16.txt
Fall: TESTING_ML_v19.txt
TESTING_ML_v20.txt -78 822 4.7132685049761385
Fall: TESTING_ML_v21.txt
Fall: TESTING_ML_v22.txt
Fall: TESTING_ML_v23.txt
Fall: TESTING_ML_v25.txt
Fall: TESTING_ML_v28.txt
Fall: TESTING_ML_v29.txt
Fall: TESTING_ML_v30.txt
Fall: TESTING_ML_v40.txt
Fall: TESTING_ML_v49.txt
Fall: TESTING_ML_v50.txt
Fall: TESTING_ML_v51.txt
Fall: TESTING_ML_v52.txt
Fall: TESTING_ML_v53.txt
Fall: TESTING_ML_v54.txt
Fall: TESTING_ML_v55.txt
Fall: TESTING_ML_v56.txt
Fall: TESTING_ML_v57.txt
Fall: TESTING_ML_v58.txt
Fall: TESTING_ML_v59.txt
Fall: TESTING_ML_v8.txt
Fall: TESTING_ML_v9.txt
Same floor Fall: TESTING_ML_BB_v100.txt
Same floor Fall: TESTING_ML_BB_v101.txt
Same floor Fall: TESTING_ML_BB_v69.txt
Same floor Fall: TESTING_ML_BB_v70.txt
Same floor Fall: TESTING_ML_BB_v71.txt
Same floor Fall: TESTING_ML_BB_v72.txt
Same floor Fall: TESTING_ML_BB_v73.txt
TESTING_ML_BB_v74.txt -20 880 17.71

In [87]:
from collections import Counter
cnt =  Counter(y)
cnt

Counter({1: 966, 0: 24, 2: 23})

In [59]:
X = np.array(x)
Y = np.array(y)
X = X.reshape(-1,t*2,6,1)
print(X.shape,Y.shape)


(1013, 900, 6, 1) (1013,)


In [61]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=32)
from collections import Counter
cnt =  Counter(y_test)
print(cnt)
orig_shape = X_train.shape

wX = X_train.reshape(-1,t*12)
print(X_train.shape,Y.shape,wX.shape)

Counter({1: 195, 2: 5, 0: 3})
(810, 900, 6, 1) (1013,) (810, 5400)


In [62]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
over = SMOTE(sampling_strategy='not majority')

steps = [('o',over)]
pipeline = Pipeline(steps=steps)
# transform the dataset
new_train_X, new_train_Y = pipeline.fit_resample(wX, y_train)

In [63]:
new_train_X = new_train_X.reshape((-1, t*2, 6, 1))
print(new_train_X.shape,new_train_Y.shape)

(2313, 900, 6, 1) (2313,)


In [64]:
y_train_cat = keras.utils.to_categorical(new_train_Y)
y_test_cat = keras.utils.to_categorical(y_test)

In [65]:
from collections import Counter
cnt =  Counter(new_train_Y)
cnt

Counter({1: 771, 0: 771, 2: 771})

In [66]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten
model = Sequential()
 
model.add(Conv2D(10, kernel_size=(50,3), activation='relu', input_shape=(t*2,6,1),strides=30))

model.add(Flatten())

model.add(Dense(3, activation='softmax'))
adam = tf.keras.optimizers.Adam(learning_rate=0.001,)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 29, 1, 10)         1510      
                                                                 
 flatten_2 (Flatten)         (None, 290)               0         
                                                                 
 dense_2 (Dense)             (None, 3)                 873       
                                                                 
Total params: 2,383
Trainable params: 2,383
Non-trainable params: 0
_________________________________________________________________


In [67]:
model.fit(new_train_X, y_train_cat, validation_data=(X_test, y_test_cat), epochs=20,batch_size=128)


Epoch 1/20
19/19 [==============================] - 1s 21ms/step - loss: 0.6380 - accuracy: 0.7812 - val_loss: 0.1007 - val_accuracy: 0.9852
Epoch 2/20
19/19 [==============================] - 0s 10ms/step - loss: 0.2068 - accuracy: 0.9810 - val_loss: 0.0646 - val_accuracy: 0.9754
Epoch 3/20
19/19 [==============================] - 0s 10ms/step - loss: 0.0585 - accuracy: 0.9987 - val_loss: 0.0332 - val_accuracy: 0.9852
Epoch 4/20
19/19 [==============================] - 0s 9ms/step - loss: 0.0223 - accuracy: 0.9991 - val_loss: 0.0241 - val_accuracy: 0.9901
Epoch 5/20
19/19 [==============================] - 0s 9ms/step - loss: 0.0128 - accuracy: 0.9991 - val_loss: 0.0194 - val_accuracy: 0.9951
Epoch 6/20
19/19 [==============================] - 0s 10ms/step - loss: 0.0092 - accuracy: 0.9991 - val_loss: 0.0152 - val_accuracy: 0.9951
Epoch 7/20
19/19 [==============================] - 0s 11ms/step - loss: 0.0070 - accuracy: 0.9991 - val_loss: 0.0133 - val_accuracy: 0.9951
Epoch 8/20
19/1

In [68]:
results = model.evaluate(X_test, y_test_cat, batch_size=128)
print(X_test.shape[0])

2/2 [==============================] - 0s 4ms/step - loss: 0.0018 - accuracy: 1.0000
203


In [69]:
from sklearn.metrics import confusion_matrix

predictions = model.predict(X, batch_size=128, verbose=0)
classes_x=(np.argmax(predictions,axis=1))

true_classes = Y

c = confusion_matrix(Y, classes_x)
print('Confusion matrix:\n', c)
print('Accuracy:{}%'.format((c[0, 0] + c[1, 1]) * 100 / (c[0, 1] + c[0, 0] + c[1, 1] + c[1, 0])))
print('Sensitivity:{}%'.format(c[0, 0]*100 / (c[0, 1] + c[0, 0])))
print('Specificity:{}%'.format(c[1, 1]*100 / (c[1, 1] + c[1, 0])))


Confusion matrix:
 [[ 24   0   0]
 [  0 966   0]
 [  0   0  23]]
Accuracy:100.0%
Sensitivity:100.0%
Specificity:100.0%


In [81]:
from tinymlgen import port
open('../model/model.h', "w").write((port(model, optimize=False)))/1024

INFO:tensorflow:Assets written to: C:\Users\soods\AppData\Local\Temp\tmppf4rpslr\assets


INFO:tensorflow:Assets written to: C:\Users\soods\AppData\Local\Temp\tmppf4rpslr\assets


68.4052734375

In [29]:
model.save('../model')

INFO:tensorflow:Assets written to: ../model\assets


INFO:tensorflow:Assets written to: ../model\assets
